# Projeto 2 - Ciência dos Dados

Nome: Antônio Fonseca

Nome: José Bechara

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [9]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [10]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***@JosAntnioBecha2***

In [11]:
# #Dados de autenticação do twitter:

# #Coloque aqui o identificador da conta no twitter: @JosAntnioBecha2 

# #leitura do arquivo no formato JSON
# with open('auth.pass') as fp:    
#     data = json.load(fp)

# #Configurando a biblioteca. Não modificar
# auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
# auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [12]:
#Produto escolhido:
produto = 'tesla'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

#### Capturando os dados do twitter:
** (Já evitando mensagens repetidas) **

In [13]:
# #Cria um objeto para a captura
# api = tweepy.API(auth)

# #Inicia a captura, para mais detalhes: ver a documentação do tweepy
# i = 1
# msgs = []
# for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
#     # Evitando mensagens repitidas:
#     if msg not in msgs: 
#         msgs.append(msg.full_text.lower())
#         i += 1
#     if i > n:
#         break

# #Embaralhando as mensagens para reduzir um possível viés
# shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [14]:
# #Verifica se o arquivo não existe para não substituir um conjunto pronto
# if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
#     #Abre o arquivo para escrita
#     writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

#     #divide o conjunto de mensagens em duas planilhas
#     dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
#     dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

#     dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
#     dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

#     #fecha o arquivo
#     writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

#### Limpando o DataFrame

In [15]:
# fonte:
# https://pt.stackoverflow.com/questions/217832/como-retirar-caractere-especial-e-ponto-de-coluna-string-de-um-data-frame
def deletar_caracteres(coluna):
    coluna = coluna.replace("https://","").replace("."," ").replace(":"," ").replace(","," ").replace("'"," ")\
    .replace('"', " ").replace("#","").replace("-","").replace("•", "").replace("\n\n", " ").replace("\n", " ")\
    .replace("—", " ").replace("rt", "").replace("?", " ").replace("!", " ").replace("/","").replace(";", " ")\
    .replace("{", "").replace("}", "").replace("|","").replace("(", "").replace(")", "").replace("_", "")\
    .replace("__", "").replace("\\","")
    return coluna

In [16]:
Treinamento = pd.read_excel(produto + ".xlsx", sheet_name= "Treinamento")
Teste = pd.read_excel(produto +".xlsx", sheet_name= "Teste")

In [17]:
Treinamento['Treinamento'] = Treinamento['Treinamento'].apply(deletar_caracteres)

In [18]:
relevante = Treinamento[Treinamento["Classificação"]==1]
tx_relevante = " ".join(relevante.Treinamento)
tx_relevante = tx_relevante.lower()

In [19]:
irrelevante= Treinamento[Treinamento["Classificação"]==2]
tx_irrelevante = " ".join(irrelevante.Treinamento)
tx_irrelevante = tx_irrelevante.lower()

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [20]:
serie_relevante = pd.Series(tx_relevante.split())
#serie_relevante

In [21]:
tabela_relevante = serie_relevante.value_counts()
d_lista_relev= tabela_relevante[1]
###tabela_relevante

In [22]:
tabela_relevante_relativa = serie_relevante.value_counts(True)
#tabela_relevante_relativa

In [23]:
serie_irrelevante = pd.Series(tx_irrelevante.split())
#serie_irrelevante

In [24]:
tabela_irrelevante = serie_irrelevante.value_counts()
d_lista_irrelev= tabela_irrelevante[1]
###tabela_irrelevante

In [25]:
tabela_irrelevante_relativa = serie_irrelevante.value_counts(True)
# tabela_irrelevante_relativa

In [26]:
def probabilidade(palavra): #2991
    if palavra in tabela_relevante:
        k = tabela_relevante[palavra]
    else:
        k = 0
    if palavra in tabela_irrelevante:
        J = tabela_irrelevante[palavra]
    else:
        J = 0
    prob_relev =( k +1)/(len(serie_relevante) + d_lista_relev)
    prob_irrelev =(J + 1)/(len(serie_irrelevante) + d_lista_irrelev) 
    return [prob_relev, prob_irrelev]

In [27]:
def prob_frase(frase):
    f = deletar_caracteres(frase)
    f = f.split()
    coeficiente_relevante = 1
    coeficiente_irrelevante = 1
    for e in f:
        coeficiente_relevante *= probabilidade(e)[0]
        coeficiente_irrelevante *= probabilidade(e)[1]
    return coeficiente_relevante >= coeficiente_irrelevante

print (prob_frase("se quiseres compreender o segredo do universo, pense em energia, frequência e vibração. tesla"))

False


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [28]:
k = 0
for e in Teste["Teste"]:
    Teste["Simulação"][k]= prob_frase(e)

KeyError: 'Simulação'

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**